In [ ]:
import requests
import pandas as pd
from math import ceil
from collections import defaultdict

PAGE_SIZE = 100
NB_DAYS = [1, 7, 30, 90, 360]
NB_RATES_PER_DAY=3

TICKER="SNX_USDT"
# mexc api: https://mxcdevelop.github.io/apidocs/contract_v1_en/#get-contract-risk-fund-balance-history
# note: Rate limit:20 times/2 seconds

df = pd.DataFrame()  # create new dataframe

nb_pages = ceil((NB_DAYS[-1]*NB_RATES_PER_DAY)/PAGE_SIZE)   #  e.g. 360 * 3 / 100 = 1080 => 11 pages of 100

for i in range(1, nb_pages+1):  # i from page_num 1 to x non-inclusive
    url= f"https://contract.mexc.com/api/v1/contract/funding_rate/history?symbol={TICKER}&page_num={i}&page_size=100"
    print(f"sending request to: '{url}'")
    rq = requests.get(url)
    rp = rq.json()
    df = df.append(pd.json_normalize(rp['data']['resultList']))

len_df = df.shape[0]
print(f"number of rows: {len_df}")

In [ ]:
df['time'] = pd.to_datetime(df['settleTime'], unit='ms')

pd.set_option('display.min_rows', 1000)
pd.set_option('display.max_rows', 20)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None) 

df = df.reset_index(drop=True)


In [11]:
result = {'Name': TICKER, 'Last': df.iloc[0]['fundingRate']}

for i in NB_DAYS:
    
    idx = i*NB_RATES_PER_DAY
    colname_sum = f'{i}d sum'
    colname_avg = f'{i}d avg'

    if len_df >= idx:        
        df[colname_sum]=df['fundingRate'].rolling(idx).sum()
        df[colname_avg]=df['fundingRate'].rolling(idx).mean()
        result[colname_sum] = df.iloc[idx-1][colname_sum]        
        result[colname_avg] = df.iloc[idx-1][colname_avg]
    else:
        result[colname_sum] = float('NaN')
        result[colname_avg] = float('NaN')


print(f">> last funding rate time: {df.iloc[0]['time']}")
dfr = pd.DataFrame([result])
dfr

>> last funding rate time: 2022-08-31 16:00:00


,Name,Last,1d sum,1d avg,7d sum,7d avg,30d sum,30d avg,90d sum,90d avg,360d sum,360d avg
0,SNX_USDT,-0.000266,-0.001987,-0.000662,-0.008232,-0.000392,-0.020745,-0.00023,-0.058808,-0.000218,0.044772,0.000041
